# __Commonly used querries__

This notebook contains:
* __Querring everything filtering using one column ['=']__
* __Querring everything filtering using two or more columns ['=']__
* __Querring specific data from specific hexs ('IN' tupple) and setting index__
* __Querring by filtering similar data ['LIKE'], for example for data starting with something [{data}%]. Data ending with something would be [%{data}] AND using WKT Geometry.__

## __Import libraries__

In [1]:
from pathlib import Path

current_path = Path().resolve()

for parent in current_path.parents:
    if parent.name == "accesibilidad-urbana":
        project_root = parent
        break

print(project_root)

/home/jovyan/accesibilidad-urbana


In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(project_root))
if module_path not in sys.path:
    print(f"Path found.")
    sys.path.append(module_path)
    import aup
else:
    print(f"Path NOT found.")
    import aup

Path found.


## __Notebook config__

In [3]:
res = 8
city = 'Aguascalientes'

## __Load data__

### __Load data__ - Everything filtering using one column __['=']__

#### __Example:__ Select the hexgrid for Aguascalientes.

In [13]:
hex_schema = 'hexgrid'
hex_table = f'hexgrid_{res}_city_2020'
query = f"SELECT * FROM {hex_schema}.{hex_table} WHERE \"city\" = '{city}\'"
hex_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(hex_gdf.crs)
print(hex_gdf.shape)
hex_gdf.head(2)

epsg:4326
(2088, 6)


,hex_id_8,geometry,CVEGEO,NOMGEO,city,type
0,88498e3289fffff,"POLYGON ((-102.16756 21.82626, -102.16297 21.8...",01001,Aguascalientes,Aguascalientes,rural
1,88498eaf5bfffff,"POLYGON ((-102.25389 21.70855, -102.24930 21.7...",01001,Aguascalientes,Aguascalientes,rural


### __Load data__ - Everything filtering using two or more columns __['=']__

#### __Example:__ Select NDVI res 8 hexs for Aguascalientes.

In [8]:
ndvi_schema = 'raster_analysis'
ndvi_table = 'ndvi_analysis_hex'

query = f"SELECT * FROM {ndvi_schema}.{ndvi_folder} WHERE \"city\" = \'{city}\' AND \"res\"={res}"
ndvi_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(ndvi_gdf.crs)
print(ndvi_gdf.shape)
ndvi_gdf.head(2)

epsg:4326
(497, 17)


,hex_id,geometry,res,ndvi_mean,ndvi_std,ndvi_median,ndvi_max,ndvi_min,ndvi_diff,ndvi_tend,ndvi_2018,ndvi_2019,ndvi_2020,ndvi_2021,ndvi_2022,ndvi_2023,city
0,884988c925fffff,"POLYGON ((-102.28685 22.05204, -102.28225 22.0...",8,0.353244,0.141792,0.324724,0.564735,0.214358,0.350377,-0.003216,0.443614,0.396490,0.399436,0.399499,0.254128,0.226298,Aguascalientes
1,884988c965fffff,"POLYGON ((-102.28945 22.02819, -102.28485 22.0...",8,0.248166,0.116135,0.219866,0.426775,0.141360,0.285415,-0.001990,0.314617,0.263361,0.290980,0.289999,0.175155,0.154886,Aguascalientes


### __Load data__ - __Specific data__ from specific hexs __('IN' tupple)__ and __setting index__

#### __Example:__ Load specific columns for the previously loaded hexs in Aguascalientes.

In [12]:
hex_ids_tuple = tuple(hex_gdf.hex_id_8.unique())

censo_folder = 'sociodemografico' 
censo_table = 'censo_inegi_20_ageb_hex'

query = f"SELECT hex_id, vivtot, tvivparhab, geometry FROM {censo_folder}.{censo_table} WHERE hex_id IN {str(hex_ids_tuple)}"
hex_censo = aup.gdf_from_query(query, geometry_col="geometry", index_col="hex_id")

# Show
print(hex_censo.crs)
print(hex_censo.shape)
hex_censo.head(2)

epsg:4326
(296, 3)


,vivtot,tvivparhab,geometry
hex_id,,,
88498e3639fffff,2303.4253,2046.1935,"POLYGON ((-102.27184 21.89588, -102.26725 21.8..."
88498e3657fffff,2167.7020,1704.0713,"POLYGON ((-102.28972 21.87858, -102.28512 21.8..."


### __Load data__ - Filtering similar data __['LIKE']__, for example for __data starting with something [{data}%]__. Data ending with something would be [%{data}] AND using __WKT Geometry__.

#### __Example:__ Load DENUE's industry for the previously loaded hex area in Aguascalientes.

In [19]:
# Download industrias (Codes starting with 31, 32 and 33) from DENUE for Aguascalientes's municipalities.

# Aguascalientes municipalities dissolved (From previously loaded hex_gdf)
poly_wkt = hex_gdf.dissolve().geometry.to_wkt()[0]
# Industries starting code
industries_codes = tuple([31,32,33])

# Iterate over each code looking for codes starting with the previous tupple, and where the points intersect with the municipalities
industries_gdf = gpd.GeoDataFrame()
denue_schema = 'denue'
denue_table = 'denue_23_point'
for industry_code in industries_codes:
    query = f"SELECT * FROM {denue_schema}.{denue_table} WHERE ST_Intersects(geometry, \'SRID=4326;{poly_wkt}\') AND \"codigo_act\" LIKE \'{industry_code}%\'"
    industries_tmp = aup.gdf_from_query(query, geometry_col='geometry')
    industries_gdf = pd.concat([industries_gdf, industries_tmp],ignore_index = True, axis = 0)

# Show
print(list(industries_gdf.codigo_act.unique()))
print(industries_gdf.crs)
print(industries_gdf.shape)
industries_gdf.head(2)

['311612', '314912', '312112', '311520', '311813', '311910', '311812', '311830', '312149', '312113', '315229', '316999', '314120', '313210', '316211', '314991', '315192', '315223', '311922', '311611', '311423', '311820', '311513', '311110', '313310', '313230', '311613', '314911', '315224', '316991', '315225', '311340', '313220', '316219', '312111', '315999', '311213', '311930', '311411', '311421', '311614', '311999', '316214', '311993', '312142', '311350', '311511', '315110', '313240', '315991', '311230', '313112', '311940', '316212', '315222', '315210', '311214', '311422', '315221', '312120', '311212', '314992', '313113', '327320', '322210', '321999', '326110', '321910', '323119', '327121', '327991', '327420', '325610', '321992', '322299', '325510', '326193', '325211', '325412', '323111', '326120', '325620', '321920', '326150', '326212', '325993', '326194', '327211', '326140', '326192', '322230', '327219', '326191', '326198', '325310', '326130', '322220', '327330', '327215', '326199',

,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,...,ageb,manzana,telefono,correoelec,www,tipounieco,latitud,longitud,fecha_alta,geometry
0,8499866.0,01005311612000063000000000U2,EMPACADORA DE CARNES ALSA,CARNES ALSA SA DE CV,311612,"Corte y empacado de carne de ganado, aves y ot...",11 a 30 personas,CARRETERA,LOCALIDAD 800,None,...,0035,800.0,4499731704,CARNESALSA@OUTLOOK.COM,None,Fijo,21.954420,-102.29128,2019-11,POINT (-102.29128 21.95442)
1,7461549.0,01005313320000012000000000U7,LONAS SUAREZ,None,314912,Confección de productos de textiles recubierto...,6 a 10 personas,PROLONGACION,INDEPENDENCIA,CALLE,...,0035,9.0,4494609689,LONAS_SUAREZ@HOTMAIL.COM,None,Fijo,21.955645,-102.29568,2019-11,POINT (-102.29568 21.95564)
